## 1- Importing Libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## 2- Setting up Face Detector

In [ ]:
def visualize(input, faces, thickness=2):
    if faces[1] is not None:
        for idx, face in enumerate(faces[1]):
            coords = face[:-1].astype(np.int32)
            cv2.rectangle(input, (coords[0], coords[1]), (coords[0]+coords[2], coords[1]+coords[3]), (0, 255, 0), thickness)
            cv2.circle(input, (coords[4], coords[5]), 2, (255, 0, 0), thickness)
            cv2.circle(input, (coords[6], coords[7]), 2, (0, 0, 255), thickness)
            cv2.circle(input, (coords[8], coords[9]), 2, (0, 255, 0), thickness)
            cv2.circle(input, (coords[10], coords[11]), 2, (255, 0, 255), thickness)
            cv2.circle(input, (coords[12], coords[13]), 2, (0, 255, 255), thickness)


detector = cv2.FaceDetectorYN.create(
    "models/face/face_detection_yunet_2022mar.onnx",
    "",
    (320, 320),
    0.8,
    0.3,
    5000
)

image1 = cv2.imread("images/akhavan.jpg")
img1 = image1.copy()
img1Width = int(img1.shape[1])
img1Height = int(img1.shape[0])
img1 = cv2.resize(img1, (img1Width, img1Height))

detector.setInputSize((img1Width, img1Height))
faces1 = detector.detect(img1)

assert faces1[1] is not None, 'Cannot find a face in {}'.format(args.image1)
# Draw results on the input image
visualize(img1, faces1)
# Save results if save is true

image2 = cv2.imread("images/alireza-in-workshop.jpg")
img2 = image2.copy()


detector.setInputSize((img2.shape[1], img2.shape[0]))
faces2 = detector.detect(img2)
assert faces2[1] is not None, 'Cannot find a face in {}'.format(args.image2)
visualize(img2, faces2)

plt.figure(figsize=[24,8])
plt.subplot(1,2,1);plt.imshow(img1[...,::-1]);plt.title("Image1");
plt.subplot(1,2,2);plt.imshow(img2[...,::-1]);plt.title("Image2");


## 3- Align Images

In [ ]:
recognizer = cv2.FaceRecognizerSF.create(
"model/face/face_recognition_sface_2021dec.onnx","")

face1_align = recognizer.alignCrop(image1, faces1[1][0])
face2_align = recognizer.alignCrop(image2, faces2[1][0])

plt.figure(figsize=[10,8])
plt.subplot(1,2,1);plt.imshow(face1_align[...,::-1]);plt.title("face1");
plt.subplot(1,2,2);plt.imshow(face2_align[...,::-1]);plt.title("face2");

## 4- Extract Features

In [ ]:
face1_feature = recognizer.feature(face1_align)
face2_feature = recognizer.feature(face2_align)

print(face1_feature.shape)
print(face2_feature.shape)

## 5- Finding Similarity

- FaceRecognizerSF_FR_NORM_L2

In [ ]:
l2_similarity_threshold = 1.128

l2_score = recognizer.match(face1_feature, face2_feature, cv2.FaceRecognizerSF_FR_NORM_L2)

msg = 'different identities'
if l2_score <= l2_similarity_threshold:
    msg = 'the same identity'
    
print('They have {}. NormL2 Distance: {}, threshold: {} (lower value means higher similarity, min 0.0).'.format(msg, l2_score, l2_similarity_threshold))

- FaceRecognizerSF_FR_COSINE

In [ ]:
cosine_similarity_threshold = 0.363

cosine_score = recognizer.match(face1_feature, face2_feature, cv2.FaceRecognizerSF_FR_COSINE)

msg = 'different identities'
if cosine_score >= cosine_similarity_threshold:
    msg = 'the same identity'
    
print('They have {}. Cosine Similarity: {}, threshold: {} (higher value means higher similarity, max 1.0).'.format(msg, cosine_score, cosine_similarity_threshold))